# Introduction

# Equations

# Numerical Methods

In [11]:
#EAS 4610 proj
#crank-nicolson method 

# import packages
import numpy as np
import matplotlib.pyplot as plt
import pylab as py
import os
import math
from scipy.sparse import spdiags
#import xarray as xr

# parameters 
k = 2 #m/d
n = 0.2 #effective porosity 
S = 1e-6 #storage coefficient 
H = 45 #m
zb = -45 #bottom of the aquifer
zt = 0 #top of the aquifer
L = 1000 #m, length of cross section
W = 1 #width of cross section normal to the plane of flow
vs = 0.025 #dimensionless density 
# ditch is located 590 m from the coastline at x = 0
h0 = 0.05 #m
N = 0 #infiltration of freshwater

tau = k*vs*(zt-zb)
T = k*(zt-zb)

# defining boundary conditions
Q0 = 0.1 #m**2/d - outflow, uniform flow towards the coast
Q0p = 0.05 #m**2/d - outflow when water is pumped from the ditch
zeta0 = -Q0/(k*vs) #m - boundary condition at the coast - top 
zeta0p = -Q0p/(k*vs) 
# at x=0, h0 = 0.05 m and zeta0 = -2 m
# at t=0, Q = -0.05 m**2/d
zetaf = -h0/vs  #steady state condition, end of simulation


# discretizing
x = np.linspace(0,L,L+1)
dx = 10 #m
t = np.linspace(0,36500,200)
dt = 183  #half a year in days

#courant number 
# no diffusion number?
D = 1
CDz = tau*dt/(n*dx**2)
CDh = T*dt/(D*dx**2) #rename to S, currently D as fake storage coeff

#n should be the length of the spatial dimension
#n = len(x)


'''
data_hi1 = np.array([-0.5*CD*np.ones(L+1), (1+CD)*np.ones(L+1), -0.5*CD*np.ones(L+1)])
diags1 = np.array([-1,0,1])
hi1 = spdiags(data_hi1,diags1,L-1,L-1).toarray()
#above coeff matrix: is it needed? prob not

data_hi = np.array([0.5*CD*np.ones(L+1), (1-CD)*np.ones(L+1), 0.5*CD*np.ones(L+1)])
diags2 = np.array([-1,0,1])
hi = spdiags(data_hi,diags2,L-1,L-1).toarray()


data_zetai1 = np.array([-0.5*CD*np.ones(L+1), (1+CD)*np.ones(L+1), -0.5*CD*np.ones(L+1)])
diags3 = np.array([-1,0,1])
zetai1 = spdiags(data_zetai1,diags3,L-1,L-1).toarray()

data_zetai = np.array([0.5*CD*np.ones(L+1), (1-CD)*np.ones(L+1), 0.5*CD*np.ones(L+1)])
diags4 = np.array([-1,0,1])
zetai = spdiags(data_zetai,diags4,L-1,L-1).toarray()
'''
data1 = np.array([CDh*np.ones(L+1), (1-2*CDh)*np.ones(L+1), CDh*np.ones(L+1)])
diags1 = np.array([-1, 0, 1])
M = spdiags(data1, diags1,L-1,L-1).toarray()

data2 = np.array([CDh*np.ones(L+1), (1-CDh)*np.ones(L+1), CDh*np.ones(L+1)])
diags2 = np.array([-1, 0, 1])
M2 = spdiags(data2, diags2,L-1,L-1).toarray()

data3 = np.array([CDz*np.ones(L+1), (1-2*CDz/vs)*np.ones(L+1), CDz*np.ones(L+1)])
diags3 = np.array([-1, 0, 1])
A = spdiags(data3, diags3,L-1,L-1).toarray()

data4 = np.array([CDz*np.ones(L+1), (1-CDz/vs)*np.ones(L+1), CDz*np.ones(L+1)])
diags4 = np.array([-1, 0, 1])
B = spdiags(data4, diags4,L-1,L-1).toarray()

M2[0,:] = 0
M[0,:] = 0
M[0,0] = 1

hi1 = np.zeros((L-1,200))
zetai1 = np.zeros((L-1,200))
zeta = np.zeros((L-1,200))  #given values matrix for zeta
hmat = np.zeros((L-1,200))  #given values matrix for h


#hi1[:,0] = 10 ##CHANGE THIS VALUE, JUST A PLACEHOLDER, INITIAL CONDITION HERE
hmat[0,:] = h0
zeta[0,:] = zeta0
zeta[-1,:] = zetaf

#print(len(zeta))
#print(len(zeta[0]))


#print((hmat))
for i in range(0,len(t)-1):
    hi1[:,i+1] = M@hmat[:,i] + M2@zetai1[:,i] #+ N
    zetai1[:,i+1] = A@zeta[:,i] + B@hi1[:,i+1] #+ dt/n
    
 #   hi1[:,i+1] = np.matmul(M,hmat[:,i]) + np.matmul(M2,zetai1[:,i]) #+ N
  #  zetai1[:,i+1] = np.matmul(A,zeta[:,i]) + np.matmul(B,hi1[:,i+1]) #+ dt/n
  #  zetai1[:,i+1] = np.linalg.inv(hi1)@(hi@zetai1[:,i]+(0.5*CD)*zeta[:,i]-(-0.5*CD)*zeta[:,i+1])
  #  hi1[:,i+1] = np.linalg.inv(zetai1)@(zetai@hi1[:,i]+(0.5*CD)*hmat[:,i]-(-0.5*CD)*hmat[:,i+1])

Mf = np.zeros((L+1,200))   #final matrix
Mf[0,:] = hmat[0,:]
Mf[1:-1] = hi1[0:,:]
Mf[-1,:] = zeta[-1,:]

'''
print(Mf)
print(hi1) 
print("Zetai1")
print(zetai1)
'''
#plt.contourf(t,x,Mf)

print(zetai1)

'''   
print(np.size(zetai1))
print(np.size(zeta))
print(len(zetai1))
print(len(zetai1[0]))
print(zetai1)

#creating the known matrices
#M**(k+1)
bins = np.array([-1/2*Cd*np.ones(n),(1+Cd)*np.ones(n),-1/2*Cd*np.ones(n)])
diags = np.array([-1,0,1])
M_k1 = spdiags(bins,diags,n,n).toarray()

#M**k
bins2 = np.array([1/2*Cd*np.ones(n),(1-Cd)*np.ones(n),1/2*Cd*np.ones(n)])
M_k = spdiags(bins2,diags,n,n).toarray()

#A
A = np.ones(n,len(t))
A[1,:] = zeta0_a
A[-1,:] = zeta0_b
'''

<ipython-input-11-ab4b6891ac2b>:112: RuntimeWarning: overflow encountered in matmul
  hi1[:,i+1] = M@hmat[:,i] + M2@zetai1[:,i] #+ N
<ipython-input-11-ab4b6891ac2b>:113: RuntimeWarning: overflow encountered in matmul
  zetai1[:,i+1] = A@zeta[:,i] + B@hi1[:,i+1] #+ dt/n
<ipython-input-11-ab4b6891ac2b>:113: RuntimeWarning: invalid value encountered in matmul
  zetai1[:,i+1] = A@zeta[:,i] + B@hi1[:,i+1] #+ dt/n


[[ 0.00000000e+00  3.42041306e+03  3.51385410e+07 ...             nan
              nan             nan]
 [ 0.00000000e+00 -6.81343312e+03 -1.42737911e+09 ...             nan
              nan             nan]
 [ 0.00000000e+00  1.69538062e+02  9.81524137e+08 ...             nan
              nan             nan]
 ...
 [ 0.00000000e+00  0.00000000e+00  1.68789552e+07 ...             nan
              nan             nan]
 [ 0.00000000e+00 -4.11750000e+01 -4.62871067e+08 ...             nan
              nan             nan]
 [ 0.00000000e+00  3.29200000e+03  4.60127826e+08 ...             nan
              nan             nan]]


'   \nprint(np.size(zetai1))\nprint(np.size(zeta))\nprint(len(zetai1))\nprint(len(zetai1[0]))\nprint(zetai1)\n\n#creating the known matrices\n#M**(k+1)\nbins = np.array([-1/2*Cd*np.ones(n),(1+Cd)*np.ones(n),-1/2*Cd*np.ones(n)])\ndiags = np.array([-1,0,1])\nM_k1 = spdiags(bins,diags,n,n).toarray()\n\n#M**k\nbins2 = np.array([1/2*Cd*np.ones(n),(1-Cd)*np.ones(n),1/2*Cd*np.ones(n)])\nM_k = spdiags(bins2,diags,n,n).toarray()\n\n#A\nA = np.ones(n,len(t))\nA[1,:] = zeta0_a\nA[-1,:] = zeta0_b\n'

# Results

# Figures 

# Video Walkthrough

# References
Bakker, M. (1970, January 1). Simple groundwater flow models for seawater intrusion: Semantic scholar. undefined. Retrieved November 27, 2022, from https://www.semanticscholar.org/paper/SIMPLE-GROUNDWATER-FLOW-MODELS-FOR-SEAWATER-Bakker/1ac46ef8707dda37496dd907e3dc9e3b0d52c4bc 